In [1]:
from parser import public_construct, public_match, public_match_exact

In [2]:
from utils import SocketReader, ReaderWrapper

In [3]:
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey, X25519PublicKey
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDFExpand, HKDF
from cryptography.hazmat.primitives.hmac import HMAC
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives.asymmetric import ec

In [4]:
try:
    sock.close()
except:
    pass

sock = SocketReader("www.google.com", 443, 'tcp')

sock.connect()

In [5]:
client_hello_spec = """
8==1?24@client_hello(
    16@version
    -32B@random
    -8@session_id()
    -16@suites(...16)
    -8@compression()
    -16@extensions(
        ...(
            16==0?16@ext_server_name(
                16@data(
                    8@name_type
                    -16@host_name()
                )
            )
            :16==5?16@ext_status_request(
                8@status_type
                -16@responder_id_list
                -16@request_extensions
            )
            :16==10?16@ext_supported_curves(
                16@data(...16)
            )
            :16==11?16@ext_supported_points(
                8@supported_points()
            )
            :16==35?16@ext_session_tickets()
            :16==13?16@ext_signature_algorithms(
                16@data(...16)
            )
            :16==50?16@ext_signature_algorithms_cert(
                16@data(...16)
            )
            :16==0xff01?16@ext_renegotiation_info(
                8@secure_renegotitation()
            )
            :16==16?16@ext_alpn(
                16@alpn_protocols(...8())
            )
            :16==43?16@ext_suppoerted_versions(
                8@supported_versions(...16)
            )
            :16==44?16@ext_cookie(
                16@cookie()
            )
            :16==51?16@ext_key_share(
                16@key_shares(
                    ...(
                        16@group
                        -16@data()
                    )
                )
            )
            :16==18?16@ext_sct(
                16@scts(
                    ...16@data()
                )
            )
            :16==24?16@ext_token_binding(

            )
            :16==42?16@empty()
            :16==45?16@ext_psk_modes(
                8@psk_modes()
            )
            :16==41?16@ext_pre_shared_key(
                16@psk_identities(
                    ...(
                        16@label()
                        -32@obfuscated_ticket_age
                    )
                )
                -16@psk_binders(
                    ...8()
                )
            )
            :16@selector-16@raw_extension()
        )
    )
)"""

client_hello_packet_spec = """
8==22?16@tls_version-16@tls_handshake(%s)
"""%client_hello_spec

In [6]:
private_key = X25519PrivateKey.generate()

public_key = private_key.public_key().public_bytes(serialization.Encoding.Raw, serialization.PublicFormat.Raw)

In [7]:
transcript = hashes.Hash(hashes.SHA256())

In [8]:
client_hello = {
    'client_hello': {
        'version': 0x0303,
        'random': 0xab6618b2bf3898950c1f4820897760317c348c3b9c8288a05e436d4ba4631d9d,
        'session_id': 0x130262b1ed623d15dae118905cde48936edcf578fa422d946c5285423d6234e6,
        'suites': [0x1301],
        'compression': 0x00,
        'extensions': [
            {
                'ext_server_name': {
                    'data': {
                        'name_type': 0x00, 'host_name': "www.google.com"
                    }
                }, '_selector': 0x0000
            },
            {
                'ext_status_request': {
                    'status_type': 0x01, 'responder_id_list': 0x0000, 'request_extensions': 0x0000
                },
                '_selector': 0x0005
            },
            {
                'ext_supported_curves': {
                    'data': [0x001d]
                }, '_selector': 0x000a
            },
            {
                'ext_supported_points': {
                    'supported_points': 0x00
                }, '_selector': 0x000b
            },
            {
                'ext_signature_algorithms': {
                    'data': [0x0403]
                }, '_selector': 0x000d
            },
            {
                'ext_renegotiation_info': {
                    'secure_renegotitation': ''
                }, '_selector': 0xff01
            },
            {
                'ext_alpn': {
                    'alpn_protocols': ["http/1.1"]
                }, '_selector': 0x0010
            },
            {
                'ext_sct': '', '_selector': 0x0012
            },
            {
                'ext_suppoerted_versions': {
                    'supported_versions': [0x0304, 0x0303, 0x0302, 0x0301]
                },
                '_selector': 0x002b
            },
            {
                'ext_key_share': {
                    'key_shares': [
                        {
                            'group': 0x001d, 'data': public_key
                        }
                    ]
                },
                '_selector': 0x0033
            }
        ]
    },
    '_selector': 0x01
}

client_hello_packet = {
    "tls_version": 0x0301,
    "tls_handshake": client_hello,
    "_selector": 22
}

In [9]:
client_hello_encoded = public_construct(client_hello_packet_spec, client_hello_packet)
client_hello_encoded

b'\x16\x03\x01\x00\xd4\x01\x00\x00\xd0\x03\x03\xabf\x18\xb2\xbf8\x98\x95\x0c\x1fH \x89w`1|4\x8c;\x9c\x82\x88\xa0^CmK\xa4c\x1d\x9d \x13\x02b\xb1\xedb=\x15\xda\xe1\x18\x90\\\xdeH\x93n\xdc\xf5x\xfaB-\x94lR\x85B=b4\xe6\x00\x02\x13\x01\x01\x00\x00\x85\x00\x00\x00\x13\x00\x11\x00\x00\x0ewww.google.com\x00\x05\x00\x05\x01\x00\x00\x00\x00\x00\n\x00\x04\x00\x02\x00\x1d\x00\x0b\x00\x02\x01\x00\x00\r\x00\x04\x00\x02\x04\x03\xff\x01\x00\x01\x00\x00\x10\x00\x0b\x00\t\x08http/1.1\x00\x12\x00\x00\x00+\x00\t\x08\x03\x04\x03\x03\x03\x02\x03\x01\x003\x00&\x00$\x00\x1d\x00 =\xcc\x13h\xe8<\xf8\xf3!\xf8\x138\x158\x98\xff\x89\x87\xac\xa6\x16b^\x07jE\xe1\x97\xa0\xdd\x85g'

In [10]:
transcript.update(client_hello_encoded[5:])

In [11]:
sock.send(client_hello_encoded)

In [12]:
def read_tls(sock):
    header = bytearray(sock.read_len(5))
    l = (header[3] << 8) + header[4]
    body = sock.read_len(l)
    return header + body

In [13]:
server_hello_encoded = read_tls(sock)

In [14]:
transcript.update(server_hello_encoded[5:])

In [15]:
server_hello_spec = """
8==2?24@server_hello(
    16@version
    -32B@random
    -8@session_id()
    -16@suite
    -8@compression
    -16@extensions(
        ...(
            16==5?16@ext_status_request(
                8@status_type
                -16@responder_id_list
                -16@request_extensions
            )
            :16==11?16@ext_supported_points(
                8@supported_points()
            )
            :16==35?16@ext_session_tickets()
            :16==0xff01?16@ext_renegotiation_info(
                8@secure_renegotitation()
            )
            :16==16?16@ext_alpn(
                16@alpn_protocols(8())
            )
            :16==18?16@ext_sct(
                16@scts(
                    ...16@data()
                )
            )
            :16==43?16@ext_suppoerted_versions(
                16@supported_versions
            )
            :16==44?16@ext_cookie(
                16@cookie()
            )
            :16==51?16@ext_key_share(
                16@group
                -16@data()
            )
            :16==41?16@ext_pre_shared_key(
                16@label()
            )
            :16==16?16@ext_alpn(
                16@alpn(
                    8@data()
                )
            )
            :16@selector-16@raw_extension()
        )
    )
)
"""


server_hello_packet_spec = """
 8==22?16@tls_version-16@tls_handshake(%s)
:8==20?16@tls_version-16@change_cypher()
:8==21?16@tls_version-16@alert()
:8==23?16@tls_version-16@application_data()
"""%server_hello_spec


In [16]:
server_hello = public_match_exact(server_hello_packet_spec, server_hello_encoded)

In [17]:
server_hello

{'tls_version': 0x0303,
 '_selector': 0x16,
 'tls_handshake': {'server_hello': {'version': 0x0303,
   'random': 0xf649fef16f2bf1274b4572507cabc2fb0eb3250e893ac43efb0d0b95d69ffafe,
   'session_id': 0x130262b1ed623d15dae118905cde48936edcf578fa422d946c5285423d6234e6,
   'suite': 0x1301,
   'compression': 0x00,
   'extensions': [{'ext_key_share': {'group': 0x001d,
      'data': 0x213427092b5e244878008e28c657819c48130cda61ae91f36570f7cb6f8b7a6a},
     '_selector': 0x0033},
    {'ext_suppoerted_versions': {'supported_versions': 0x0304},
     '_selector': 0x002b}]},
  '_selector': 0x02}}

In [18]:
server_public_key = None
for extension in server_hello['tls_handshake']['server_hello']['extensions']:
    if 'ext_key_share' in extension:
        print("found public key")
        server_public_key = extension['ext_key_share']['data'].bytes()
        break
    else:
        raise RuntimeError("unable to find server public key")


found public key


In [19]:
shared_key = private_key.exchange(X25519PublicKey.from_public_bytes(server_public_key))
print("calculated shared key", shared_key)

calculated shared key b'\x91b8\xab_\x1d\xc0M\xf3t@\x0e\xff\xb3\xef\xe4\xd0\xf6\x18B\xfe\x01\x05Y\xaf{\xb6\xf1U-\xc0S'


In [20]:
server_change_cipher_encoded = read_tls(sock)

In [21]:
server_change_cipher = public_match_exact(server_hello_packet_spec, server_change_cipher_encoded)
server_change_cipher

{'tls_version': 0x0303, '_selector': 0x14, 'change_cypher': 0x01}

In [22]:
sock.send(server_change_cipher_encoded)

In [23]:
def get_info(label, length, context=b''):
    return b'\x00' + bytearray((length, len(b'tls13 '+label),)) + b'tls13 ' + label + bytearray((len(context),)) + context

def extract(secret, salt):
    return HKDF(hashes.SHA256(), 32, salt, b'')._extract(secret)    
    
def derive(secret, label, prev=None):
    context = get_hash(prev)
    return expand(secret, label, 32, context)
    
def expand(secret, label, l, context=b''):
    return HKDFExpand(hashes.SHA256(), l, get_info(label, l, context)).derive(secret)

def get_hash(prev):
    if prev is None:
        prev = hashes.Hash(hashes.SHA256())
    c = prev.copy()
    return c.finalize()

def finished_hash(secret, prev=None):
    finished_key = expand(secret, b'finished', 32)
    context = get_hash(prev)
    h = HMAC(finished_key, hashes.SHA256())
    h.update(context)
    return h.finalize()
    

In [24]:
early_secret = extract(b'\0'*32, None)

extractor = derive(early_secret, b"derived")

handshake_secret = extract(shared_key, extractor)

In [25]:
server_handshake_secret = derive(handshake_secret, b"s hs traffic", transcript)

server_handshake_key = expand(server_handshake_secret, b'key', 16)
server_handshake_nonce = (expand(server_handshake_secret, b'iv', 12), 0)

In [26]:
client_handshake_secret = derive(handshake_secret, b"c hs traffic", transcript)

client_handshake_key = expand(client_handshake_secret, b'key', 16)
client_handshake_nonce = (expand(client_handshake_secret, b'iv', 12), 0)

In [27]:
server_cert_encrypted_encoded = read_tls(sock)

In [28]:
server_cert_encrypted = public_match_exact(server_hello_packet_spec, server_cert_encrypted_encoded)
server_cert_encrypted

{'tls_version': 0x0303,
 '_selector': 0x17,
 'application_data': 0x921699d459b71083f942f47c71ae23df0f11423edfb0295dfa804b7d635c89dac351dfd3350fe201559c4b7e10cb5a4beca797dd6b1d054906498fd41c95c30d789690c9587478999560acc777e5aec7f8c424a3cf0c976eb7f5eabcc1a393464f9bdc8e3bed3e95406a58a2d66312e1dee37a69473ea44a4f73b656d93387a604810da6487840b2367edef030628f23b9721e292b2dcd0b865211dd23afb9a3f4123f3bd753047cfff7be5b6d7e9bfe7ec1fd4e86c799acedb1284f4287f89d1cab66244ed0b70e545d01b4964951c8aa756c99e93c2012603172913da8f9984fb4ca53424ae274c222b56f362fac1d6f16bb9cc0b27da3cfe0b385df29c8a7e790474f3aef5e63e720b8a5d196ccacc481e5bbb608da73a316dc3b37a94829b47f5e8b37c824fece2a855b68ae1bd31f93de54264ade2bfcfffe2c2fddd87723db638a42b1c464488da39d56fade90d2cd049615c11daaf4b442d070364181a48167d8f186ee7cd0491a1c458c9162b1711e664ec1e99b9c15c0d4bf5f832bf27c80be3bf170de2106ca49f5c059b2c8a4631e94652b51cbab9992f4d8ec745a8aee501fc585c9628d6f42cf934dc35883d494a565243c55673ad5f4f0c1b50690b6203eb32f3bc050e09298be1bc24227e

In [29]:
def update_nonce(nonce):
    return (nonce[0], nonce[1]+1)

def get_nonce(nonce):
    return bytes(a ^ b for a, b in zip(nonce[0], nonce[1].to_bytes(len(nonce[0]), 'big')))

In [30]:
def decrypt_packet(packet, key, nonce):
    packet = bytes(packet)
    cipher = AESGCM(key)
    all_server_cert_packet = cipher.decrypt(get_nonce(nonce), packet[5:], packet[:5])
    server_cert_packet, server_cert_packet_suffix = all_server_cert_packet[:-1], all_server_cert_packet[-1]
    return server_cert_packet, server_cert_packet_suffix, update_nonce(nonce)

def encrypt_packet(packet, suffix, key, nonce):
    app_header_spec = "8@packet_type-16@tls_version-16@application_data_length"

    raw_packet = packet + suffix

    header_encoded = public_construct(app_header_spec, {
        "packet_type": 23,
        "tls_version": 0x0303,
        "application_data_length": len(raw_packet) + 16,
    })

    
    cipher = AESGCM(key)
    encrypted_app_data = cipher.encrypt(get_nonce(nonce), raw_packet, header_encoded)
    packet = header_encoded + encrypted_app_data
    return packet, update_nonce(nonce)

In [31]:
server_cert_packet, server_cert_packet_suffix, server_handshake_nonce = decrypt_packet(server_cert_encrypted_encoded, server_handshake_key, server_handshake_nonce)
print(server_cert_packet_suffix)
server_cert_packet

22


b'\x08\x00\x00\x11\x00\x0f\x00\x10\x00\x0b\x00\t\x08http/1.1\x0b\x00\x0e\xe2\x00\x00\x0e\xde\x00\x04Z0\x82\x04V0\x82\x03>\xa0\x03\x02\x01\x02\x02\x10I|M\xc1\x05\x88O\x96\x12f\xe5\x85\x9eT\x17F0\r\x06\t*\x86H\x86\xf7\r\x01\x01\x0b\x05\x000;1\x0b0\t\x06\x03U\x04\x06\x13\x02US1\x1e0\x1c\x06\x03U\x04\n\x13\x15Google Trust Services1\x0c0\n\x06\x03U\x04\x03\x13\x03WR20\x1e\x17\r240603073532Z\x17\r240826073531Z0\x191\x170\x15\x06\x03U\x04\x03\x13\x0ewww.google.com0Y0\x13\x06\x07*\x86H\xce=\x02\x01\x06\x08*\x86H\xce=\x03\x01\x07\x03B\x00\x044\xd1\xe9\x05\xf5\x8f.\x968\xd3\xa2\xc77H\xbb\xea\xe9\x10\x1d/q\xff\x94\xc2\xa0\x93n\xf0\x9dw \xd8\x95\x949@\x9d\x8d%\xf5!\x07\x83LDu\xbcY\xaa\x9b\xf5\x9d?\x17\x1e\xb6G\xfc\xd6\xbc\xb5\xdav\xcb\xa3\x82\x02A0\x82\x02=0\x0e\x06\x03U\x1d\x0f\x01\x01\xff\x04\x04\x03\x02\x07\x800\x13\x06\x03U\x1d%\x04\x0c0\n\x06\x08+\x06\x01\x05\x05\x07\x03\x010\x0c\x06\x03U\x1d\x13\x01\x01\xff\x04\x020\x000\x1d\x06\x03U\x1d\x0e\x04\x16\x04\x14\xdf\xa3\xbc\x18\x00\xb3\x9e\xe9\xb

In [32]:
multiple_packets = server_cert_packet

packet_len = (multiple_packets[1] << 16) + (multiple_packets[2] << 8) + multiple_packets[3]
encrypted_extensions_packet, multiple_packets = multiple_packets[:4+packet_len], multiple_packets[4+packet_len:]

packet_len = (multiple_packets[1] << 16) + (multiple_packets[2] << 8) + multiple_packets[3]
certificates_packet, multiple_packets = multiple_packets[:4+packet_len], multiple_packets[4+packet_len:]

packet_len = (multiple_packets[1] << 16) + (multiple_packets[2] << 8) + multiple_packets[3]
certificate_verify_packet, multiple_packets = multiple_packets[:4+packet_len], multiple_packets[4+packet_len:]

packet_len = (multiple_packets[1] << 16) + (multiple_packets[2] << 8) + multiple_packets[3]
finished_message_packet, multiple_packets = multiple_packets[:4+packet_len], multiple_packets[4+packet_len:]

In [33]:
handshake_app_data_spec = """
    8==8?24@encrypted_extenstions(
        16@data(
            16@alpn
            -16@data(
                16@data(
                    8@data()
                )
            )
        )
    )
    :8==11?24@certificates_data(
        8@certificate_request_context-24@certificates_data(
            ...(
                24@certificate()
                -16@extensions(
                    16==5?16@status_request(
                        8@status_type_ocsp
                        -24@ocsp_staple(
                        )
                    )
                    :16==18?16@sct(
                        16@data(
                            ...16@data()
                        )
                    )
                )
            )
        )
    )
    :8==15?24@certificate_verify(
        16@signature_algorithm-16@signature()
    )
    :8==20?24@finished_verify_data()
    :8==22?24@certificate_status(
        8@status_type
        -24@response()
    )
    :8@selector-24@data()
"""

In [34]:
encrypted_extensions = public_match_exact(handshake_app_data_spec, encrypted_extensions_packet)
encrypted_extensions

{'encrypted_extenstions': {'data': {'alpn': 0x0010,
   'data': {'data': {'data': "http/1.1"}}}},
 '_selector': 0x08}

In [35]:
server_cert = public_match_exact(handshake_app_data_spec, certificates_packet)
server_cert

{'certificates_data': {'certificate_request_context': 0x00,
  'certificates_data': [{'certificate': 0x308204563082033ea0030201020210497c4dc105884f961266e5859e541746300d06092a864886f70d01010b0500303b310b3009060355040613025553311e301c060355040a1315476f6f676c65205472757374205365727669636573310c300a06035504031303575232301e170d3234303630333037333533325a170d3234303832363037333533315a3019311730150603550403130e7777772e676f6f676c652e636f6d3059301306072a8648ce3d020106082a8648ce3d0301070342000434d1e905f58f2e9638d3a2c73748bbeae9101d2f71ff94c2a0936ef09d7720d8959439409d8d25f52107834c4475bc59aa9bf59d3f171eb647fcd6bcb5da76cba38202413082023d300e0603551d0f0101ff04040302078030130603551d25040c300a06082b06010505070301300c0603551d130101ff04023000301d0603551d0e04160414dfa3bc1800b39ee9bd615d77b127a354350d544a301f0603551d23041830168014de1b1eed7915d43e3724c321bbec34396d42b230305806082b06010505070101044c304a302106082b060105050730018615687474703a2f2f6f2e706b692e676f6f672f777232302506082b06010505073002861968747470

In [36]:
import OpenSSL.crypto

first_cert = None
for cert in server_cert['certificates_data']['certificates_data']:
    x509 = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_ASN1, cert['certificate'].bytes())
    if first_cert is None:
        first_cert = x509
    print(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_TEXT, x509).decode('utf-8')) 

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            49:7c:4d:c1:05:88:4f:96:12:66:e5:85:9e:54:17:46
        Signature Algorithm: sha256WithRSAEncryption
        Issuer: C=US, O=Google Trust Services, CN=WR2
        Validity
            Not Before: Jun  3 07:35:32 2024 GMT
            Not After : Aug 26 07:35:31 2024 GMT
        Subject: CN=www.google.com
        Subject Public Key Info:
            Public Key Algorithm: id-ecPublicKey
                Public-Key: (256 bit)
                pub:
                    04:34:d1:e9:05:f5:8f:2e:96:38:d3:a2:c7:37:48:
                    bb:ea:e9:10:1d:2f:71:ff:94:c2:a0:93:6e:f0:9d:
                    77:20:d8:95:94:39:40:9d:8d:25:f5:21:07:83:4c:
                    44:75:bc:59:aa:9b:f5:9d:3f:17:1e:b6:47:fc:d6:
                    bc:b5:da:76:cb
                ASN1 OID: prime256v1
                NIST CURVE: P-256
        X509v3 extensions:
            X509v3 Key Usage: critical
                Digital Signature
 

In [37]:
cert_pub = first_cert.get_pubkey().to_cryptography_key()

In [38]:
transcript.update(encrypted_extensions_packet)
transcript.update(certificates_packet)

In [39]:
cert_verify_message = public_match_exact(handshake_app_data_spec, certificate_verify_packet)
cert_verify_message

{'certificate_verify': {'signature_algorithm': 0x0403,
  'signature': 0x3045022100f1ef17d5711c688b0d01f95ee85be7da5f0a05d2257865af63c473c5e75cbabb02203b7403aed07c15f5ba78df2975f88b57af23ceefe4930e38f2482fa325291cc7},
 '_selector': 0x0f}

In [40]:
sign_data = b'\x20' * 64 + \
    b'TLS 1.3, server CertificateVerify\x00' + \
    get_hash(transcript)

signature = cert_verify_message['certificate_verify']['signature'].bytes()

cert_pub.verify(signature, sign_data, ec.ECDSA(hashes.SHA256()))
print("certificate verify succeeded")

certificate verify succeeded


In [41]:
transcript.update(certificate_verify_packet)

In [42]:
finished_message = public_match_exact(handshake_app_data_spec, finished_message_packet)
finished_message

{'finished_verify_data': 0xc7f40efd2b3cb38e07f47217764237ee0b9be1cee30dd5691c730aecd7ed4ba1,
 '_selector': 0x14}

In [43]:
if finished_hash(server_handshake_secret, transcript) == finished_message['finished_verify_data'].bytes():
    print("matching finished hash")
else:
    raise RuntimeError("not matching finished hash")

matching finished hash


In [44]:
transcript.update(finished_message_packet)

In [45]:
client_finished_hash = finished_hash(client_handshake_secret, transcript)
client_verify_unencrypted = public_construct(handshake_app_data_spec, {'finished_verify_data': client_finished_hash, '_selector': 0x14})

In [46]:
client_verify_packet, client_handshake_nonce = encrypt_packet(
    client_verify_unencrypted, b'\x16', client_handshake_key, client_handshake_nonce
)

In [47]:
sock.send(client_verify_packet)

In [48]:
master_secret = extract(b'\0' * 32, derive(handshake_secret, b"derived"))
server_app_secret = derive(master_secret, b"s ap traffic", transcript)
server_app_key = expand(server_app_secret, b'key', 16)
server_app_nonce = (expand(server_app_secret, b'iv', 12), 0)
client_app_secret = derive(master_secret, b"c ap traffic", transcript)
client_app_key = expand(client_app_secret, b'key', 16)
client_app_nonce = (expand(client_app_secret, b'iv', 12), 0)

In [49]:
transcript.update(client_verify_packet)

In [50]:
http_req = b"""GET / HTTP/1.1
Host: www.google.com

"""
packet, client_app_nonce = encrypt_packet(
    http_req, b'\x17', client_app_key, client_app_nonce
)

In [51]:
sock.send(packet)

In [52]:
http_resp = read_tls(sock)

In [53]:
data, packet_type, server_app_nonce = decrypt_packet(http_resp, server_app_key, server_app_nonce)
(data, packet_type, server_app_nonce)

(b'HTTP/1.1 200 OK\r\nDate: Wed, 26 Jun 2024 19:38:46 GMT\r\nExpires: -1\r\nCache-Control: private, max-age=0\r\nContent-Type: text/html; charset=ISO-8859-1\r\nContent-Security-Policy-Report-Only: object-src \'none\';base-uri \'self\';script-src \'nonce-kqlvBJHCflREQSM_IbhCfQ\' \'strict-dynamic\' \'report-sample\' \'unsafe-eval\' \'unsafe-inline\' https: http:;report-uri https://csp.withgoogle.com/csp/gws/other-hp\r\nP3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."\r\nServer: gws\r\nX-XSS-Protection: 0\r\nX-Frame-Options: SAMEORIGIN\r\nSet-Cookie: AEC=AQTF6Hz-xLEeKuYABFlb23W4pdq4NLL2KRJjX0Y74AT563hk1wU_NkFZow; expires=Mon, 23-Dec-2024 19:38:46 GMT; path=/; domain=.google.com; Secure; HttpOnly; SameSite=lax\r\nSet-Cookie: __Secure-ENID=20.SE=hSKvJq9I3MKIQuc5t5xoNdp7mHTvpX5rOamo9Ke0iun-PttRqK70ZwDDXVjgLeYcHQBvK81yaTBkwzJxqfJeoZUo3Ot-GBfEcCesDOvz-0TLqvnkVwFXf14mmTpanvHmT26RVqFvZQYPTFw4l9M77sflZpxX4PtaHS_C-2fClJpF367XjWTbB9P4unIDaJc5-66ItrVoQYWUzjbK; expires=Sun, 27-Jul-

In [54]:
class HttpsReader(ReaderWrapper):
    def _read(self):
        global server_app_nonce
        http_resp = read_tls(sock)
        data, packet_type, server_app_nonce = decrypt_packet(http_resp, server_app_key, server_app_nonce)
        print("got body", data[:100])
        return data

In [55]:
http_reader = HttpsReader(initial_value=data)

In [56]:
http_reader.read_line()

b'HTTP/1.1 200 OK\r\n'

In [57]:
content_length = None
chunked = False

while True:
    header = http_reader.read_line()
    print(header)
    if header == b'\r\n':
        break
    if header.lower().startswith(b'content-length:'):
        content_length = int(i.split(':').strip())
        print('got content length', content_length)
    elif header.lower().startswith(b'transfer-encoding: chunked'):
        chunked = True
        print('got chunked')


b'Date: Wed, 26 Jun 2024 19:38:46 GMT\r\n'
b'Expires: -1\r\n'
b'Cache-Control: private, max-age=0\r\n'
b'Content-Type: text/html; charset=ISO-8859-1\r\n'
b"Content-Security-Policy-Report-Only: object-src 'none';base-uri 'self';script-src 'nonce-kqlvBJHCflREQSM_IbhCfQ' 'strict-dynamic' 'report-sample' 'unsafe-eval' 'unsafe-inline' https: http:;report-uri https://csp.withgoogle.com/csp/gws/other-hp\r\n"
b'P3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."\r\n'
b'Server: gws\r\n'
b'X-XSS-Protection: 0\r\n'
b'X-Frame-Options: SAMEORIGIN\r\n'
b'Set-Cookie: AEC=AQTF6Hz-xLEeKuYABFlb23W4pdq4NLL2KRJjX0Y74AT563hk1wU_NkFZow; expires=Mon, 23-Dec-2024 19:38:46 GMT; path=/; domain=.google.com; Secure; HttpOnly; SameSite=lax\r\n'
b'Set-Cookie: __Secure-ENID=20.SE=hSKvJq9I3MKIQuc5t5xoNdp7mHTvpX5rOamo9Ke0iun-PttRqK70ZwDDXVjgLeYcHQBvK81yaTBkwzJxqfJeoZUo3Ot-GBfEcCesDOvz-0TLqvnkVwFXf14mmTpanvHmT26RVqFvZQYPTFw4l9M77sflZpxX4PtaHS_C-2fClJpF367XjWTbB9P4unIDaJc5-66ItrVoQYWUzjbK; expires=Sun, 2

In [58]:
body = b''

In [59]:
while chunked:
    print("reading chunks")
    length = http_reader.read_line()
    print("length line", length)
    l = int(length[:-2], 16)

    ch = http_reader.read_len(l)
    print("got chunk", len(ch), ch[:100])
    body += ch
    http_reader.read_line()
    if l == 0:
        break

reading chunks
length line b'4f8e\r\n'
got body b'leg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="kqlvBJHCflREQSM_I'
got body b"436,120,3,96,2,54,138,693,204,1519,2,21191038,365637,3,980,5,9211,4,8136,1381,1183,906',kBL:'jpKz',k"
got body b' g=n.length;n[g]=a;a.onerror=a.onload=a.onabort=function(){delete n[g]};a.src=c}};google.logUrl=func'
got body b':transparent;position:absolute;top:-999px;visibility:hidden;z-index:998;right:0}.gbto #gbs{backgroun'
got body b'dius=5)";opacity:1\\0/;top:-4px\\0/;left:-6px\\0/;right:5px\\0/;bottom:4px\\0/}.gbma{position:relative;to'
got body b'o .gbgt .gbtb2{border-top-width:0}.gbtb .gbts{background:url(https://ssl.gstatic.com/gb/images/b_8d5'
got body b'(https://ssl.gstatic.com/gb/images/b_8d5afc09.png);_background:url(https://ssl.gstatic.com/gb/images'
got body b'v{background:#fff;border-bottom:1px solid #bebebe;-moz-box-shadow:0 2px 4px rgba(0,0,0,.12);-o-box-s'
got body b':0;white-space:nowrap;width:100%}.gbmpala

In [60]:
from IPython.core.display import display, HTML
display(HTML(body.decode("latin1")))

/tmp/ipykernel_1112368/449586190.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


,,Geavanceerd zoeken


In [61]:
sock.close()